# Working files
- added diagonal to unitlist

In [1]:
rows = 'ABCDEFGHI'
cols = '123456789'

In [2]:
# 
assignments = []

def assign_value(values, box, value):  #? dict, box, value
    """
    Please use this function to update your values dictionary!
    Assigns a value to a given box. If it updates the board record it.
    """

    # Don't waste memory appending actions that don't actually change any values
    if values[box] == value:
        return values

    values[box] = value
    if len(value) == 1:
        assignments.append(values.copy())
    return values


In [3]:
def cross(a, b):
    return [s+t for s in a for t in b]

In [4]:

boxes = cross(rows, cols)
row_units = [cross(r, cols) for r in rows]
column_units = [cross(rows, c) for c in cols]
square_units = [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]

#DIAGONALS
diag_topL_bottomR = [[row_units[i][i] for i in range(len(row_units))]]  # Diagonal Top left to bottom right
diag_topR_bottomL = [[row_units[i][len(row_units)-1-i] for i in range(len(row_units))]]  # Diagonal from top right bottom left

In [5]:
# lets add diags to unitlist
#unitlist = row_units + column_units + square_units

unitlist = row_units + column_units + square_units + diag_topL_bottomR + diag_topR_bottomL

In [6]:
units = dict((s, [u for u in unitlist if s in u]) for s in boxes)
peers = dict((s, set(sum(units[s],[]))-set([s])) for s in boxes)

In [7]:
def display(values):
    """
    Display the values as a 2-D grid.
    Input: The sudoku in dictionary form
    Output: None
    """
    width = 1+max(len(values[s]) for s in boxes)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '')
                      for c in cols))
        if r in 'CF': print(line)
    return

In [8]:
def clean_values(grid):
    """Convert grid string into {<box>: <value>} dict with '.' value for empties.

    Args:
        grid: Sudoku grid in string form, 81 characters long
    Returns:
        Sudoku grid in dictionary form:
        - keys: Box labels, e.g. 'A1'
        - values: Value in corresponding box, e.g. '8', or '.' if it is empty.
    """
    assert len(grid) == 81, "Input grid must be a string of length 81 (9x9)"
    return dict(zip(boxes, grid))

# WARNING! We've modified this function to return '123456789' instead of '.' for boxes with no value.
# Look at the explanation above in the text.

In [9]:
def grid_values(grid):
    """Convert grid string into {<box>: <value>} dict with '123456789' value for empties.

    Args:
        grid: Sudoku grid in string form, 81 characters long
    Returns:
        Sudoku grid in dictionary form:
        - keys: Box labels, e.g. 'A1'
        - values: Value in corresponding box, e.g. '8', or '123456789' if it is empty.
    """
    values = []
    all_digits = '123456789'
    for c in grid:
        if c == '.':
            values.append(all_digits)
        elif c in all_digits:
            values.append(c)
    assert len(values) == 81
    return dict(zip(boxes, values))

In [10]:
# goes 1st

def eliminate(values):
    """Eliminate values from peers of each box with a single value.

    Go through all the boxes, and whenever there is a box with a single value,
    eliminate this value from the set of values of all its peers.

    Args:
        values: Sudoku in dictionary form.
    Returns:
        Resulting Sudoku in dictionary form after eliminating values.
    """
    solved_values = [box for box in values.keys() if len(values[box]) == 1]
    for box in solved_values:
        digit = values[box]
        for peer in peers[box]:
            values[peer] = values[peer].replace(digit,'')
    return values

In [11]:
# goes 2nd
def only_choice(values):
    """Finalize all values that are the only choice for a unit.

    Go through all the units, and whenever there is a unit with a value
    that only fits in one box, assign the value to this box.

    Input: Sudoku in dictionary form.
    Output: Resulting Sudoku in dictionary form after filling in only choices.
    """
    for unit in unitlist:
        for digit in '123456789':
            dplaces = [box for box in unit if digit in values[box]]
            if len(dplaces) == 1:
                values[dplaces[0]] = digit
    return values

In [12]:
def naked_twins(values):
    """Eliminate values using the naked twins strategy.
    Args:
        values(dict): a dictionary of the form {'box_name': '123456789', ...}

    Returns:
        the values dictionary with the naked twins eliminated from peers.
    """
    # Find all instances of naked twins
    # Eliminate the naked twins as possibilities for their peers
    # get boxes with only two values
    twin_values = [box for box in values.keys() if len(values[box]) == 2]
    # from those boxes get only with matching values
    nk =[[tw1,tw2] for tw1 in twin_values for tw2 in peers[tw1] if set(values[tw1]) == set(values[tw2])]
    
    for i in range(len(nk)):
        twin_pack1 = nk[i][0]  # nk[0][0]
        twin_pack2 = nk[i][1]  # nk[1][1]
        tp1_peers = peers[twin_pack1]  # peers of twin_pack1
        tp2_peers = peers[twin_pack2]  # peers of twin_pack2
        twin_peers = tp1_peers & tp2_peers  # peer boxes in tp1_peers AND tp2_peers ONLY
        
        for peer_values in twin_peers:  # loop over those boxes
            if len(values[peer_values])> 2:  # if val is more than 2
                for remove_duplicate in values[twin_pack1]:
                    # replace naked twin val with empty string
                    values[peer_values] = values[peer_values].replace(remove_duplicate,'')
    return values

In [13]:
def reduce_puzzle(values):
    """
    Iterate eliminate() and only_choice(). If at some point, there is a box with no available values, return False.
    If the sudoku is solved, return the sudoku.
    If after an iteration of both functions, the sudoku remains the same, return the sudoku.
    Input: A sudoku in dictionary form.
    Output: The resulting sudoku in dictionary form.
    """
    stalled = False

    while not stalled:
        # Check how many boxes have a determined value
        solved_values_before = len([box for box in values.keys() if len(values[box]) == 1])
        
        #NAKED TWIN???
        #values = naked_twins(values)
        # Use the Eliminate Strategy
        values = eliminate(values)
        # Use the Only Choice Strategy
        values = only_choice(values)
        # Check how many boxes have a determined value, to compare
        solved_values_after = len([box for box in values.keys() if len(values[box]) == 1])
        # If no new values were added, stop the loop.
        stalled = solved_values_before == solved_values_after
        # Sanity check, return False if there is a box with zero available values:
        if len([box for box in values.keys() if len(values[box]) == 0]):
            return False
    return values

In [14]:
# goes 3rd
def search(values):
    "Using depth-first search and propagation, try all possible values."
    # First, reduce the puzzle using the previous function
    values = reduce_puzzle(values)
    if values is False:
        return False ## Failed earlier
    if all(len(values[s]) == 1 for s in boxes):
        return values ## Solved!
    # Choose one of the unfilled squares with the fewest possibilities
    n,s = min((len(values[s]), s) for s in boxes if len(values[s]) > 1)
    # Now use recurrence to solve each one of the resulting sudokus, and
    for value in values[s]:
        new_sudoku = values.copy()
        new_sudoku[s] = value
        attempt = search(new_sudoku)
        if attempt:
            return attempt

In [15]:
easy = '..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'
medium = '4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......'
diag_sudoku_grid = '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'


game_board = grid_values(diag_sudoku_grid)
game_board_clean = clean_values(diag_sudoku_grid)



print('-------------------------------')
print("ORIGINAL BOARD")
display(game_board_clean)

print('-------------------------')
print('SOLVED BOARD')

display(search(game_board))


-------------------------------
ORIGINAL BOARD
2 . . |. . . |. . . 
. . . |. . 6 |2 . . 
. . 1 |. . . |. 7 . 
------+------+------
. . 6 |. . 8 |. . . 
3 . . |. 9 . |. . 7 
. . . |6 . . |4 . . 
------+------+------
. 4 . |. . . |8 . . 
. . 5 |2 . . |. . . 
. . . |. . . |. . 3 
-------------------------
SOLVED BOARD
2 6 7 |9 4 5 |3 8 1 
8 5 3 |7 1 6 |2 4 9 
4 9 1 |8 2 3 |5 7 6 
------+------+------
5 7 6 |4 3 8 |1 9 2 
3 8 4 |1 9 2 |6 5 7 
1 2 9 |6 5 7 |4 3 8 
------+------+------
6 4 2 |3 7 9 |8 1 5 
9 3 5 |2 8 1 |7 6 4 
7 1 8 |5 6 4 |9 2 3 


In [33]:
naked_test = {"G7": "2345678", "G6": "1236789", "G5": "23456789", "G4": "345678",
"G3": "1234569", "G2": "12345678", "G1": "23456789", "G9": "24578",
"G8": "345678", "C9": "124578", "C8": "3456789", "C3": "1234569",
"C2": "1234568", "C1": "2345689", "C7": "2345678", "C6": "236789",
"C5": "23456789", "C4": "345678", "E5": "678", "E4": "2", "F1": "1",
"F2": "24", "F3": "24", "F4": "9", "F5": "37", "F6": "37", "F7": "58",
"F8": "58", "F9": "6", "B4": "345678", "B5": "23456789", "B6":
"236789", "B7": "2345678", "B1": "2345689", "B2": "1234568", "B3":
"1234569", "B8": "3456789", "B9": "124578", "I9": "9", "I8": "345678",
"I1": "2345678", "I3": "23456", "I2": "2345678", "I5": "2345678",
"I4": "345678", "I7": "1", "I6": "23678", "A1": "2345689", "A3": "7",
"A2": "234568", "E9": "3", "A4": "34568", "A7": "234568", "A6":
"23689", "A9": "2458", "A8": "345689", "E7": "9", "E6": "4", "E1":
"567", "E3": "56", "E2": "567", "E8": "1", "A5": "1", "H8": "345678",
"H9": "24578", "H2": "12345678", "H3": "1234569", "H1": "23456789",
"H6": "1236789", "H7": "2345678", "H4": "345678", "H5": "23456789",
"D8": "2", "D9": "47", "D6": "5", "D7": "47", "D4": "1", "D5": "36",
"D2": "9", "D3": "8", "D1": "36"}



In [34]:
game_board = grid_values(diag_sudoku_grid)
game_board_clean = clean_values(diag_sudoku_grid)



print('-------------------------------')
print("ORIGINAL BOARD")
display(naked_test)

print('-------------------------')
print('SOLVED BOARD')

display(search(naked_test))

-------------------------------
ORIGINAL BOARD
 2345689   234568     7    |  34568      1      23689  |  234568   345689    2458  
 2345689  1234568  1234569 |  345678  23456789  236789 | 2345678  3456789   124578 
 2345689  1234568  1234569 |  345678  23456789  236789 | 2345678  3456789   124578 
---------------------------+---------------------------+---------------------------
    36       9        8    |    1        36       5    |    47       2        47   
   567      567       56   |    2       678       4    |    9        1        3    
    1        24       24   |    9        37       37   |    58       58       6    
---------------------------+---------------------------+---------------------------
 23456789 12345678 1234569 |  345678  23456789 1236789 | 2345678   345678   24578  
 23456789 12345678 1234569 |  345678  23456789 1236789 | 2345678   345678   24578  
 2345678  2345678   23456  |  345678  2345678   23678  |    1      345678     9    
-------------------------
SOL

In [31]:

def test_twins(values):
    twin_values = [box for box in values.keys() if len(values[box]) == 2]
    # from those boxes get only with matching values
    nk =[[tw1,tw2] for tw1 in twin_values for tw2 in peers[tw1] if set(values[tw1]) == set(values[tw2])]
    print(nk)
    
    for i in range(len(nk)):
        twin_pack1 = nk[i][0]  # nk[0][0]
        twin_pack2 = nk[i][1]  # nk[1][1]
        tp1_peers = peers[twin_pack1]  # peers of twin_pack1
        tp2_peers = peers[twin_pack2]  # peers of twin_pack2
        twin_peers = tp1_peers & tp2_peers  # peer boxes in tp1_peers AND tp2_peers ONLY
        
        for peer_values in twin_peers:  # loop over those boxes
            if len(values[peer_values])> 2:  # if val is more than 2
                for remove_duplicate in values[twin_pack1]:
                    # replace naked twin val with empty string
                    values[peer_values] = values[peer_values].replace(remove_duplicate,'')
    return values

test_twins(naked_test)

[['F2', 'F3'], ['F3', 'F2'], ['F5', 'F6'], ['F6', 'F5'], ['F7', 'F8'], ['F8', 'F7'], ['D9', 'D7'], ['D7', 'D9'], ['D5', 'D1'], ['D1', 'D5']]


{'A1': '2345689',
 'A2': '234568',
 'A3': '7',
 'A4': '34568',
 'A5': '1',
 'A6': '23689',
 'A7': '234568',
 'A8': '345689',
 'A9': '2458',
 'B1': '2345689',
 'B2': '1234568',
 'B3': '1234569',
 'B4': '345678',
 'B5': '23456789',
 'B6': '236789',
 'B7': '2345678',
 'B8': '3456789',
 'B9': '124578',
 'C1': '2345689',
 'C2': '1234568',
 'C3': '1234569',
 'C4': '345678',
 'C5': '23456789',
 'C6': '236789',
 'C7': '2345678',
 'C8': '3456789',
 'C9': '124578',
 'D1': '36',
 'D2': '9',
 'D3': '8',
 'D4': '1',
 'D5': '36',
 'D6': '5',
 'D7': '47',
 'D8': '2',
 'D9': '47',
 'E1': '567',
 'E2': '567',
 'E3': '56',
 'E4': '2',
 'E5': '68',
 'E6': '4',
 'E7': '9',
 'E8': '1',
 'E9': '3',
 'F1': '1',
 'F2': '24',
 'F3': '24',
 'F4': '9',
 'F5': '37',
 'F6': '37',
 'F7': '58',
 'F8': '58',
 'F9': '6',
 'G1': '23456789',
 'G2': '12345678',
 'G3': '1234569',
 'G4': '345678',
 'G5': '23456789',
 'G6': '1236789',
 'G7': '2345678',
 'G8': '345678',
 'G9': '24578',
 'H1': '23456789',
 'H2': '12345678',
 

In [ ]:




easy = '..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'
medium = '4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......'
diag_sudoku_grid = '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'


data = grid_values(diag_sudoku_grid)
clean_data = clean_values(diag_sudoku_grid)
#eminate = eliminate(data)
#ochoice = only_choice(data)

#eliminate_choice = eliminate(data)


#sample = eliminate(data)

In [ ]:
nk_dataset = {'A1': '1',
 'A2': '123456789',
 'A3': '4',
 'A4': '123456789',
 'A5': '9',
 'A6': '123456789',
 'A7': '123456789',
 'A8': '6',
 'A9': '8',
 'B1': '9',
 'B2': '5',
 'B3': '6',
 'B4': '123456789',
 'B5': '1',
 'B6': '8',
 'B7': '123456789',
 'B8': '3',
 'B9': '4',
 'C1': '123456789',
 'C2': '123456789',
 'C3': '8',
 'C4': '4',
 'C5': '123456789',
 'C6': '6',
 'C7': '9',
 'C8': '5',
 'C9': '1',
 'D1': '5',
 'D2': '1',
 'D3': '2379',
 'D4': '123456789',
 'D5': '123456789',
 'D6': '123456789',
 'D7': '123456789',
 'D8': '8',
 'D9': '6',
 'E1': '8',
 'E2': '123456789',
 'E3': '379',
 'E4': '6',
 'E5': '123456789',
 'E6': '123456789',
 'E7': '123456789',
 'E8': '1',
 'E9': '2',
 'F1': '6',
 'F2': '4',
 'F3': '23',
 'F4': '123456789',
 'F5': '8',
 'F6': '123456789',
 'F7': '123456789',
 'F8': '9',
 'F9': '7',
 'G1': '7',
 'G2': '8',
 'G3': '1',
 'G4': '9',
 'G5': '2',
 'G6': '3',
 'G7': '6',
 'G8': '4',
 'G9': '5',
 'H1': '4',
 'H2': '9',
 'H3': '5',
 'H4': '123456789',
 'H5': '6',
 'H6': '123456789',
 'H7': '8',
 'H8': '2',
 'H9': '3',
 'I1': '123456789',
 'I2': '6',
 'I3': '23',
 'I4': '8',
 'I5': '5',
 'I6': '4',
 'I7': '1',
 'I8': '7',
 'I9': '9'}

In [ ]:
display(nk_dataset)

In [ ]:

easy = '..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'
medium = '4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......'
diag_sudoku_grid = '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'


data = grid_values(diag_sudoku_grid)
clean_data = clean_values(diag_sudoku_grid)


def peer_twins(values):
    
    twin_values = [box for box in values.keys() if len(values[box]) == 2]
    for box in twin_values:
        digit = values[box]
        for peer in peers[box]:
            if digit == values[peer]:
                print(digit)
                print(values[peer])
                #values[peer] = values[peer].replace(digit,'')
    return values
        
    
def unit_twins(values):
    twin_values = [box for box in values.keys() if len(values[box]) == 2]
    for box in twin_values:
        digit = values[box]
        for peer in peers[box]:
            #print('box:{}, value;{}, peer:{}, peer value:{}'.format(box, digit,peer, values[peer]))
            #print(peer,values[peer])

            if digit in values[peer]:
                print('box:{}, value;{}, peer:{}, peer value:{}'.format(box, digit,peer, values[peer]))
                pass
                #print("box:{}, peer:{}. We have a match".format(box, peer))
                #print(peers[box])
                
           # print('box:{}, value;{}, peer:{}, peer value:{}'.format(box, digit,peer, values[peer]))
    #return values
            
        


In [ ]:
unit_twins(eliminate(data))

In [ ]:
nk_dataset = {'A1': '1',
 'A2': '123456789',
 'A3': '4',
 'A4': '123456789',
 'A5': '9',
 'A6': '123456789',
 'A7': '123456789',
 'A8': '6',
 'A9': '8',
 'B1': '9',
 'B2': '5',
 'B3': '6',
 'B4': '123456789',
 'B5': '1',
 'B6': '8',
 'B7': '123456789',
 'B8': '3',
 'B9': '4',
 'C1': '123456789',
 'C2': '123456789',
 'C3': '8',
 'C4': '4',
 'C5': '123456789',
 'C6': '6',
 'C7': '9',
 'C8': '5',
 'C9': '1',
 'D1': '5',
 'D2': '1',
 'D3': '2379',
 'D4': '123456789',
 'D5': '123456789',
 'D6': '123456789',
 'D7': '123456789',
 'D8': '8',
 'D9': '6',
 'E1': '8',
 'E2': '123456789',
 'E3': '379',
 'E4': '6',
 'E5': '123456789',
 'E6': '123456789',
 'E7': '123456789',
 'E8': '1',
 'E9': '2',
 'F1': '6',
 'F2': '4',
 'F3': '23',
 'F4': '123456789',
 'F5': '8',
 'F6': '123456789',
 'F7': '123456789',
 'F8': '9',
 'F9': '7',
 'G1': '7',
 'G2': '8',
 'G3': '1',
 'G4': '9',
 'G5': '2',
 'G6': '3',
 'G7': '6',
 'G8': '4',
 'G9': '5',
 'H1': '4',
 'H2': '9',
 'H3': '5',
 'H4': '123456789',
 'H5': '6',
 'H6': '123456789',
 'H7': '8',
 'H8': '2',
 'H9': '3',
 'I1': '123456789',
 'I2': '6',
 'I3': '23',
 'I4': '8',
 'I5': '5',
 'I6': '4',
 'I7': '1',
 'I8': '7',
 'I9': '9'}


easy = '..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'
medium = '4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......'
diag_sudoku_grid = '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'


data = grid_values(diag_sudoku_grid)
clean_data = clean_values(diag_sudoku_grid)





In [ ]:
nk_dataset = {'A1': '1',
 'A2': '123456789',
 'A3': '4',
 'A4': '123456789',
 'A5': '9',
 'A6': '123456789',
 'A7': '123456789',
 'A8': '6',
 'A9': '8',
 'B1': '9',
 'B2': '5',
 'B3': '6',
 'B4': '123456789',
 'B5': '1',
 'B6': '8',
 'B7': '123456789',
 'B8': '3',
 'B9': '4',
 'C1': '123456789',
 'C2': '123456789',
 'C3': '8',
 'C4': '4',
 'C5': '123456789',
 'C6': '6',
 'C7': '9',
 'C8': '5',
 'C9': '1',
 'D1': '5',
 'D2': '1',
 'D3': '2379',
 'D4': '123456789',
 'D5': '123456789',
 'D6': '123456789',
 'D7': '123456789',
 'D8': '8',
 'D9': '6',
 'E1': '8',
 'E2': '123456789',
 'E3': '379',
 'E4': '6',
 'E5': '123456789',
 'E6': '123456789',
 'E7': '123456789',
 'E8': '1',
 'E9': '2',
 'F1': '6',
 'F2': '4',
 'F3': '23',
 'F4': '123456789',
 'F5': '8',
 'F6': '123456789',
 'F7': '123456789',
 'F8': '9',
 'F9': '7',
 'G1': '7',
 'G2': '8',
 'G3': '1',
 'G4': '9',
 'G5': '2',
 'G6': '3',
 'G7': '6',
 'G8': '4',
 'G9': '5',
 'H1': '4',
 'H2': '9',
 'H3': '5',
 'H4': '123456789',
 'H5': '6',
 'H6': '123456789',
 'H7': '8',
 'H8': '2',
 'H9': '3',
 'I1': '123456789',
 'I2': '6',
 'I3': '23',
 'I4': '8',
 'I5': '5',
 'I6': '4',
 'I7': '1',
 'I8': '7',
 'I9': '9'}

def nakkt(values):
    twin_values = [box for box in values.keys() if len(values[box]) == 2]  # only show keys whose val is length of 2 
    
    #nk =[[box1,box2] for box1 in twin_values for box2 in peers[box1] if set(values[box1]) == set(values[box2]) ]
    # only show boxes that have the same value
    nk =[[tw1,tw2] for tw1 in twin_values for tw2 in peers[tw1] if set(values[tw1]) == set(values[tw2]) ]

   #print(nk[1][1])
    
    for i in range(len(nk)):  # 0 upto but not including nk

        box1 = nk[i][0]  # nk[0][0]
        box2 = nk[i][1]  # nk[1][1]
        print("box1:{}".format(box1))
        print('box2:{}'.format(box2))
        
        peers1 = set(peers[box1])
        print("peer of box1:{}".format(peers1))
        peers2 = set(peers[box2])
        print("peer of box2:{}".format(peers2))

        peers_int = peers1 & peers2  # boxes that share both match both
        #print("peers_int:{}".format(peers_int))
        for peer_val in peers_int:
            if len(values[peer_val])> 2:  # if val is more than 2
                for rm_val in values[box1]:
                    pass
                    #print(values[peer_val])
                   #values[peer_val] = values[peer_val].replace(rm_val,'')


       # return values
    


nakkt(nk_dataset)

In [ ]:
nk_dataset = {'A1': '1',
 'A2': '123456789',
 'A3': '4',
 'A4': '123456789',
 'A5': '9',
 'A6': '123456789',
 'A7': '123456789',
 'A8': '6',
 'A9': '8',
 'B1': '9',
 'B2': '5',
 'B3': '6',
 'B4': '123456789',
 'B5': '1',
 'B6': '8',
 'B7': '123456789',
 'B8': '3',
 'B9': '4',
 'C1': '123456789',
 'C2': '123456789',
 'C3': '8',
 'C4': '4',
 'C5': '123456789',
 'C6': '6',
 'C7': '9',
 'C8': '5',
 'C9': '1',
 'D1': '5',
 'D2': '1',
 'D3': '2379',
 'D4': '123456789',
 'D5': '123456789',
 'D6': '123456789',
 'D7': '123456789',
 'D8': '8',
 'D9': '6',
 'E1': '8',
 'E2': '123456789',
 'E3': '379',
 'E4': '6',
 'E5': '123456789',
 'E6': '123456789',
 'E7': '123456789',
 'E8': '1',
 'E9': '2',
 'F1': '6',
 'F2': '4',
 'F3': '23',
 'F4': '123456789',
 'F5': '8',
 'F6': '123456789',
 'F7': '123456789',
 'F8': '9',
 'F9': '7',
 'G1': '7',
 'G2': '8',
 'G3': '1',
 'G4': '9',
 'G5': '2',
 'G6': '3',
 'G7': '6',
 'G8': '4',
 'G9': '5',
 'H1': '4',
 'H2': '9',
 'H3': '5',
 'H4': '123456789',
 'H5': '6',
 'H6': '123456789',
 'H7': '8',
 'H8': '2',
 'H9': '3',
 'I1': '123456789',
 'I2': '6',
 'I3': '23',
 'I4': '8',
 'I5': '5',
 'I6': '4',
 'I7': '1',
 'I8': '7',
 'I9': '9'}

def unit_twins(values):
    # get boxes with only two values
    twin_values = [box for box in values.keys() if len(values[box]) == 2]
    # from those boxes get only with matching values
    nk =[[tw1,tw2] for tw1 in twin_values for tw2 in peers[tw1] if set(values[tw1]) == set(values[tw2])]
    
    for i in range(len(nk)):
        twin_pack1 = nk[i][0]  # nk[0][0]
        twin_pack2 = nk[i][1]  # nk[1][1]
        tp1_peers = peers[twin_pack1]  # peers of twin_pack1
        tp2_peers = peers[twin_pack2]  # peers of twin_pack2
        twin_peers = tp1_peers & tp2_peers  # peer boxes in tp1_peers AND tp2_peers ONLY
        
        for peer_values in twin_peers:  # loop over those boxes
            if len(values[peer_values])> 2:  # if val is more than 2
                for remove_duplicate in values[twin_pack1]:
                    # replace naked twin val with empty string
                    values[peer_values] = values[peer_values].replace(remove_duplicate,'')
    return values
    
    
    
    
unit_twins(nk_dataset)